In [1]:
import os
import numpy as np
import cv2

In [2]:
y = np.load("train_labels.npy")
X = np.load("normalized_train_rgb_data.npy")

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2, ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
import tensorflow as tf
from tensorflow.keras.regularizers import l1, l2, l1_l2
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available: 1


In [4]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(64, 64, 3))

In [5]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.8)(x)
output = Dense(10, activation= 'softmax')(x)
model = Model(inputs = base_model.input, outputs = output)

#Freeze first layers
for layer in base_model.layers[:20]:
    layer.trainable = False
    
#Unfreeze the remaining layers to allow fine-tuning
for layer in base_model.layers[30:]:
    layer.trainable = True

In [6]:
from keras.optimizers import Adam
from keras import metrics
adam = Adam(learning_rate=0.0001)
model.compile(optimizer= adam, loss='categorical_crossentropy', metrics=['accuracy', metrics.Precision(), metrics.Recall()])

In [7]:
history = model.fit(X, y, epochs=10, batch_size=128, validation_split=0.1)

Epoch 1/10
352/352 [==============================] - 62s 145ms/step - loss: 2.3930 - accuracy: 0.2025 - precision: 0.3425 - recall: 0.0274 - val_loss: 4.7325 - val_accuracy: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/10
352/352 [==============================] - 59s 168ms/step - loss: 1.6435 - accuracy: 0.4119 - precision: 0.6396 - recall: 0.2072 - val_loss: 7.1320 - val_accuracy: 0.0238 - val_precision: 0.0366 - val_recall: 0.0178
Epoch 3/10
352/352 [==============================] - 61s 173ms/step - loss: 1.4047 - accuracy: 0.4987 - precision: 0.6998 - recall: 0.3089 - val_loss: 8.1354 - val_accuracy: 0.0244 - val_precision: 0.0317 - val_recall: 0.0180
Epoch 4/10
352/352 [==============================] - 61s 174ms/step - loss: 1.2886 - accuracy: 0.5410 - precision: 0.7218 - recall: 0.3697 - val_loss: 8.4995 - val_accuracy: 0.0296 - val_precision: 0.0338 - val_recall: 0.0250
Epoch 5/10
352/352 [==============================] - 61s 174ms/step - loss: 1.1

In [8]:
model.save("WildSeekAI.keras")